# Zynq Ultrascale+ DisplayPort output

This notebook introduces using the DisplayPort output of the Zynq Ultrascale+ through PYNQ. By default we don't start X and instead provide a lower-level API to match the HDMI API we provide for the PYNQ-Z1 board.

To start with we import the `video` library which contains the `DisplayPort` class and supporting helper classes.

In [1]:
from pynq.lib.video import *

Next we create a `DisplayPort` object and configure it with the resolution and pixel format we would like.

In [2]:
displayport = DisplayPort()

displayport.configure(VideoMode(1280, 720, 24), PIXEL_RGB)

We can now test the output by setting the blue pixel of the frame in a loop and outputting the loop. From this we can see that the loop easily runs at 60 frames per second.

In [3]:
import time

start = time.time()

for i in range(600):
    frame = displayport.newframe()
    frame[:,:,0] = i % 256
    displayport.writeframe(frame)
    
end = time.time()
duration = end - start
print(f"Took {duration} seconds at {600 / duration} FPS")

Took 10.003438472747803 seconds at 59.97937625493172 FPS


It's far more exciting though to show an image from a webcam. We can do this using OpenCV. First we need to instantiate the `VideoCapture` device and set the resolution to match our frame.

For the next cell, make sure you have a webcam plugged into one of the USB ports.

In [4]:
import os
os.environ["OPENCV_LOG_LEVEL"]="SILENT"
import cv2

capture = cv2.VideoCapture(0)

capture.set(3, 1280)
capture.set(4, 720)

True

Then we can do a similar loop and record the framerate. This is slower due to the overhead in capturing frames from the camera.

In [5]:
number_frames = 150
start = time.time()

for _ in range(number_frames):
    frame = displayport.newframe()
    capture.read(frame)
    displayport.writeframe(frame)

end = time.time()
duration = end - start
print(f"Took {duration} seconds at {number_frames / duration} FPS")

Took 22.52149271965027 seconds at 6.660304530752672 FPS


Finally we need to close the devices.

In [6]:
capture.release()
displayport.close()